# Import the necessary libraries for completing the project
- cv2 for computer vision
- numpy for handling the array operartion
- os for handling the file directory

In [6]:

import cv2, numpy, os

# Loading haarcasacde algorithm for detecting the face
haar_file = 'haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier(haar_file)

dataset = 'dataset'
print('Training....')

# Taking some other variables to take data from the dataset

(images,labels,names,id) = ([],[],{},0) # here []--> means array, {}-->means set, 0 means first subdir

# Directory processing--> going to append the dataset into a single array..
# Going to get the files, their id and their names from the directory

for (subdirs, dirs, files) in os.walk(dataset): #Entering into the dataset folder
    for subdir in dirs:
        names[id] = subdir # Assign subdir to each id by its name. subdir are the user name
        subjectpath = os.path.join(dataset, subdir) #searching the path of subdir
        for filename in os.listdir(subjectpath): #Reading the files inside the subdir
            
            #path of a specific subforlder
            path = subjectpath + '/' + filename
            label = id # assign id as label. label=0 -->1st person, label=1-->2nd person
            images.append(cv2.imread(path,0)) # Taking the id->0 images into an array
            labels.append(int(label)) # Taking the label in an array
        id = id + 1

#Put images and labels into an array. Converting everything into list        
(images,labels) = [numpy.array(lis) for lis in [images,labels]]
print(images, labels)
(width,height) = (130,100)

# Initializing the classifier for recognition..

model = cv2.face.LBPHFaceRecognizer_create()
#model = cv2.face.FisherFaceRecognizer_create()

#Creating a training model..
model.train(images, labels)

#Obtaining the image from the camera
webcam = cv2.VideoCapture(0) #Initializing the camear
cnt = 0

while True:
    (_,im) = webcam.read() #Read the frame
    gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY) #Convert the frame into gray image
    faces = face_cascade.detectMultiScale(gray, 1.3, 5) # Taking the face coordinates
    
    #To obtain the face. Where the face is located
    for (x,y,w,h) in faces:
        cv2.rectangle(im,(x,y),(x+w,y+h),(255,255,0),2) #Drawing rectangle around the face
        face = gray[y:y+h,x:x+w] #Converting into gray and crop only the face part
        face_resize = cv2.resize(face,(width,height)) #Resizing

        prediction = model.predict(face_resize) #For prediction. Return the accuracy and the label id
        cv2.rectangle(im,(x,y),(x+w,y+h),(255,255,0),2) #Drawing rectangle ahape
        
        #Checking the prediction
        #Only freater than some value, then it will be a predicted person.
        #It will try to compare captured image with the database and give the prediction
        if prediction[1] < 800: #For more accuracy, have to increase this value
            cv2.putText(im,'%s - %.0f' % (names[prediction[0]],prediction[1]),(x-10,y-10),cv2.FONT_HERSHEY_PLAIN,2,(0,0,255))
            print(names[prediction[0]]) #Prediction[0] means the id
            cnt = 0
            
        else: #For unknown people. If value is found greater than 100. The person should be unknown
            cnt += 1
            cv2.putText(im,'Unknown',(x-10,y-10),cv2.FONT_HERSHEY_PLAIN,1,(0,255,0))
            if cnt > 100:
                print("Unknown Person")
                cv2.imwrite("Unknown.jpg",im) #Write the unknown image into the dataset
                cnt = 0
    cv2.imshow('FaceRecognition',im) #For showing the frame
    
    #For terminating the window
    key = cv2.waitKey(1) & 0xFF
    if key == ord ("q"):
        break
webcam.release()
cv2.destroyAllWindows()




Training....
[[[ 82  84  86 ... 115 117 118]
  [ 84  86  89 ... 118 117 118]
  [ 86  88  91 ... 116 116 117]
  ...
  [111 111 112 ... 105 104 103]
  [111 110 110 ... 106 105 104]
  [109 109 111 ... 104 103 102]]

 [[ 82  84  85 ... 115 117 119]
  [ 86  87  89 ... 113 116 118]
  [ 84  85  89 ... 117 117 119]
  ...
  [109 109 112 ... 100 102 104]
  [110 108 111 ... 101 101 101]
  [109 107 112 ... 103 100  99]]

 [[ 87  89  89 ... 119 117 117]
  [ 89  86  86 ... 119 117 117]
  [ 89  87  85 ... 117 116 116]
  ...
  [109 111 114 ...  99  99  98]
  [110 113 116 ... 100  97  96]
  [108 112 116 ... 100  95  94]]

 ...

 [[112 113 115 ... 174 173 173]
  [115 122 113 ... 173 173 173]
  [117 118 112 ... 172 174 174]
  ...
  [ 38  37  37 ...  75  74  84]
  [ 39  38  38 ...  68  73  79]
  [ 40  40  40 ...  68  67  70]]

 [[121 122 123 ... 173 172 172]
  [126 124 122 ... 173 173 173]
  [126 125 121 ... 174 173 173]
  ...
  [ 78  68  76 ...  66  70  71]
  [ 90  81  84 ...  71  68  66]
  [101  91  88 

![output](./image/output.JPG)